# LINCS CMap Chemical Perturbations Croissant Builder 🥐

Author: Ido Diamant, Ma'ayan Lab, CFDE DRC

In [1]:
import datetime
import hashlib
import json
import mlcroissant as mlc
import os
import pandas as pd
import requests
from zipfile import ZipFile

## Construct Croissant from CFDE Attribute Table

In [2]:
name='LINCS L1000 CMAP Signatures of Differentially Expressed Genes for Small Molecules'
description="Protein localization assemblies constructed from integrating AP-MS biomolecular interaction and IF imaging data"

In [3]:
short_citation = 'Duan, Nucleic Acids Res, 2014'
title = 'LINCS Canvas Browser: interactive web app to query, browse and interrogate LINCS L1000 gene expression signatures'
author = 'Duan, Q'
journal = 'Nucleic Acids Res'
year = 2014
volume = 42
pages = 'W449-W460'

cite_as=(f'@article{{{short_citation}, title={{{title}}}, author={{{author}}}, journal={{{journal}}}, year={{{year}}}, volume={{{volume}}}, pages={{{pages}}}}}')

In [4]:
creators = [
    mlc.Organization(name="The Library of Integrated Network-Based Cellular Signatures ", url="https://lincsproject.org/")
]

publishers = [
    mlc.Organization(name="Ma'ayan Lab", url="https://maayanlab.cloud/")
]

license = "https://lincsproject.org/LINCS/data/release-policy"
version = "0.1.0"
url="https://maayanlab.cloud/Harmonizome/dataset/LINCS+L1000+CMAP+Signatures+of+Differentially+Expressed+Genes+for+Small+Molecules"
date_published=datetime.date(2015, 4, 6)

### Resource

In [5]:
# Utility function to generate a SHA256 checksum for a FileObject from a URL
def get_sha256(url):
    sha256 = hashlib.sha256()
    response = requests.get(url, stream=True)
    for chunk in response.iter_content(chunk_size=65536):
        sha256.update(chunk)
    return sha256.hexdigest()

In [6]:
file_url = 'https://maayanlab.cloud/static/hdfs/harmonizome/data/lincscmapchemical/gene_attribute_matrix.txt.zip'

distribution = [
    mlc.FileObject(
        id="dataset-attribute-table-archive",
        name="dataset-attribute-table-archive",
        description="Dataset attribute table archive from Harmonizome.",
        content_url=file_url,
        encoding_formats="application/zip",
        sha256=get_sha256(file_url)
    ),
    mlc.FileObject(
        id="dataset-attribute-table",
        name="dataset-attribute-table",
        description="Dataset attribute table from Harmonizome.",
        content_url="gene_attribute_matrix.txt",
        encoding_formats="text/tab-separated-values",
        contained_in={"dataset-attribute-table-archive"}
    )
]

### Structure/Semantics

In [7]:
matrix = pd.read_csv(file_url, sep='\t', compression='zip')
display(matrix)

,GeneSym,BRD-A00267231_HEMADO_A549_24.0_h_10.0_um,BRD-A00267231_HEMADO_ASC_24.0_h_10.0_um,BRD-A00267231_HEMADO_HA1E_24.0_h_10.0_um,BRD-A00267231_HEMADO_HCC515_24.0_h_10.0_um,BRD-A00267231_HEMADO_PC3_24.0_h_10.0_um,BRD-A00267231_HEMADO_PC3_6.0_h_10.0_um,BRD-A00420644_-666_A375_6.0_h_10.0_um,BRD-A00420644_-666_A549_24.0_h_10.0_um,BRD-A00420644_-666_HCC515_6.0_h_10.0_um,...,BRD-U88878891_-666_VCAP_24.0_h_10.0_um,BRD-U88878891_-666_VCAP_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_HA1E_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_MDST8_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_PC3_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_RKO_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_SNGM_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_SNUC5_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_SW948_6.0_h_10.0_um,BRD-U94846492_Quinine hemisulfate salt monohydrate_VCAP_6.0_h_10.0_um
0,1060P11.3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A2M,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AADAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8342,ZWILCH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8343,ZWINT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8344,ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8345,ZZEF1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
fields = []
array_size = matrix.shape[0]

fields.append(
  mlc.Field(
    id="associations/gene",
    name="gene",
    description="The NCBI gene symbol",
    data_types=mlc.DataType.TEXT,
    is_array=True,
    array_shape=str(array_size),
    source=mlc.Source(
      file_object="dataset-attribute-table",
      extract=mlc.Extract(column="GeneSym")
    )
  )
)

for col in matrix.columns[1:]:
  fields.append(
    mlc.Field(
      id=f"associations/{col.replace(' ','_')}",
      name=f"associations/{col}",
      data_types=mlc.DataType.INTEGER,
      is_array=True,
      array_shape=str(array_size),
      source=mlc.Source(
        file_object="dataset-attribute-table",
        extract=mlc.Extract(column=col)
      )
    )
  )

print(len(fields))

31029


In [9]:
record_sets = [
    mlc.RecordSet(
        id="associations",
        name="associations",
        key='associations/gene',
        fields = fields
    )
]

### Assemble Dataset and Write JSON
Once we have defined each dataset layer, we can package the dataset metadata into a Croissant JSON-LD file.

In [10]:
metadata = mlc.Metadata(
    name = name,
    description = description,
    cite_as = cite_as,
    url = url,
    date_published=date_published,
    creators = creators,
    publisher = publishers,
    license = license,
    version = version,
    distribution = distribution,
    record_sets = record_sets
)

# Display any warnings/suggestions encountered by the validator when building the metadata
print(metadata.issues.report())

In [11]:
with open("lincscmapchemical.json", "w") as f:
  content = metadata.to_json()
  content = json.dumps(content, indent=4, default=str)
  f.write(content)
  f.write("\n")  # Terminate file with newline